In [ ]:
import pygame, sys
from pygame.locals import *
import random, time

# Инициализация Pygame
pygame.init()

# FPS
FPS = 60
FramePerSec = pygame.time.Clock()

# Цвета
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)

# Экран
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
DISPLAYSURF = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Racer Game")

# Фон
background = pygame.image.load("images/AnimatedStreet.png")

# Музыка
pygame.mixer.music.load("sounds/background.wav")
pygame.mixer.music.play(-1)

# Шрифты
font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
font_large = pygame.font.SysFont("Verdana", 40)
game_over = font.render("Game Over", True, BLACK)

# Глобальные переменные
SPEED_ENEMY = 5
SCORE = 0
COIN_SCORE = 0

# Таймеры
INC_SPEED = pygame.USEREVENT + 1
SPAWN_COIN = pygame.USEREVENT + 2
pygame.time.set_timer(INC_SPEED, 1000)
pygame.time.set_timer(SPAWN_COIN, 3000)  # монета раз в 3 сек

# Класс врага
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("images/Enemy.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

    def move(self):
        self.rect.move_ip(0, SPEED_ENEMY)
        if self.rect.bottom > SCREEN_HEIGHT:
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

# Класс игрока
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("images/Player.png")
        self.rect = self.image.get_rect()
        self.rect.center = (160, 520)

    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if self.rect.left > 0 and pressed_keys[K_LEFT]:
            self.rect.move_ip(-5, 0)
        if self.rect.right < SCREEN_WIDTH and pressed_keys[K_RIGHT]:
            self.rect.move_ip(5, 0)

# Класс монеты с разной ценностью и размером
class Coin(pygame.sprite.Sprite):
    def __init__(self, value):
        super().__init__()
        self.value = value
        self.image = pygame.image.load("images/coin.png")

        if self.value == 1:
            size = 20
        elif self.value == 3:
            size = 30
        else:  # 5
            size = 40

        self.image = pygame.transform.scale(self.image, (size, size))
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

    def move(self):
        self.rect.move_ip(0, SPEED_ENEMY)
        if self.rect.top > SCREEN_HEIGHT:
            self.kill()  # исчезает за экраном

# Игрок и враг
P1 = Player()
E1 = Enemy()

# Группы спрайтов
enemies = pygame.sprite.Group()
enemies.add(E1)

coins = pygame.sprite.Group()

all_sprites = pygame.sprite.Group()
all_sprites.add(P1)
all_sprites.add(E1)

# Главный игровой цикл
while True:
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

        # Генерация одной новой монеты
        if event.type == SPAWN_COIN:
            if len(coins) == 0:  # только если нет монеты на экране
                value = random.choice([1, 3, 5])
                new_coin = Coin(value)
                coins.add(new_coin)
                all_sprites.add(new_coin)

    # Повышение скорости врага в зависимости от монет
    if COIN_SCORE >= 30:
        SPEED_ENEMY = 11
    elif COIN_SCORE >= 20:
        SPEED_ENEMY = 9
    elif COIN_SCORE >= 10:
        SPEED_ENEMY = 7

    # Рендер фона
    DISPLAYSURF.blit(background, (0, 0))

    # Отображение счетов
    score_text = font_small.render("Score: " + str(SCORE), True, BLACK)
    coin_text = font_small.render("Coins: " + str(COIN_SCORE), True, BLACK)
    DISPLAYSURF.blit(score_text, (10, 10))
    DISPLAYSURF.blit(coin_text, (SCREEN_WIDTH - 100, 10))

    # Движение и отображение всех объектов
    for entity in all_sprites:
        entity.move()
        DISPLAYSURF.blit(entity.image, entity.rect)

    # Проверка на столкновение с монетой
    for coin in coins:
        if pygame.sprite.collide_rect(P1, coin):
            pygame.mixer.Sound("sounds/coin-257878.mp3").play()
            COIN_SCORE += 1  # увеличиваем только счёт монет
            SCORE += coin.value  # увеличиваем общий счёт на значение монеты
            coin.kill()

    # Проверка на столкновение с врагом
    if pygame.sprite.spritecollideany(P1, enemies):
        pygame.mixer.Sound("sounds/crash.wav").play()
        time.sleep(1)

        DISPLAYSURF.fill(RED)
        DISPLAYSURF.blit(game_over, (30, 250))
        score_text = font_large.render(f"Score: {SCORE}", True, BLACK)
        DISPLAYSURF.blit(score_text, (90, 320))
        pygame.display.update()
        time.sleep(2)

        pygame.quit()
        sys.exit()

    pygame.display.update()
    FramePerSec.tick(FPS)